In [4]:
from germansentiment import SentimentModel
import torch
import pandas as pd

In [ ]:
# Quick test
torch.cuda.empty_cache()

model = SentimentModel()
texts = [
    "Mit keinem guten Ergebniss","Das ist gar nicht mal so gut",
    "Total awesome!","nicht so schlecht wie erwartet",
    "Der Test verlief positiv.","Sie fährt ein grünes Auto."]

predictions = model.predict_sentiment(texts)
classes, probs = model.predict_sentiment(texts, output_probabilities = True)
probs

In [6]:
# Read and clean our data
df = pd.read_csv('data/comment_data_general_091022_untilarticle5874_corrected.csv',
                 index_col=0)[['comment_id', 'heading_and_text.comment']]
# display(df)

texts = df['heading_and_text.comment'].tolist()
texts_clean = [item if type(item) == str else '' for item in texts]
del df, texts
print(len(texts_clean))

1203395


In [7]:
# Get sentiment scores, processed in batches
torch.cuda.empty_cache()
model = SentimentModel()

batchsize = 100
list_probs = []
list_classes = []
for i in range(0, 200, batchsize):
    # if i % 10000 == 0: # progress check
    #     print(100*i/len(texts_clean), '%')
    classes, probs = model.predict_sentiment(texts_clean[i:i+batchsize], output_probabilities = True)
    list_probs.append(probs)
    list_classes.append(classes)

list_classes_flat = [item for sublist in list_classes for item in sublist]
list_probs_flat = [{x[0]: x[1] for x in item} for sublist in list_probs for item in sublist]

df_classes = pd.DataFrame(list_classes_flat)
df_probs = pd.DataFrame(list_probs_flat).rename({'positive': 'probs_pos',
                                                 'negative': 'probs_neg',
                                                 'neutral': 'probs_neu'},
                                                 axis=1)
display(df_classes.head())
display(df_probs.head())

/Users/Patrick/Documents/LSE/Projects/commentgap/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


,0
0,neutral
1,neutral
2,negative
3,neutral
4,neutral


,probs_pos,probs_neg,probs_neu
0,0.004558,0.020377,0.975065
1,0.000004,0.000005,0.999991
2,0.025184,0.957283,0.017533
3,0.000004,0.000008,0.999987
4,0.001303,0.004363,0.994334


In [5]:
# Write DataFrames to CSV
# df_classes.to_csv('data/output_classes.csv', index=False, encoding='utf-8')
# df_probs.to_csv('data/output_probs.csv', index=False, encoding='utf-8')